In [63]:

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.chrome.service import Service

import numpy as np
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [64]:
def create_driver():
    # Configurar las opciones del driver
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')

    # Crear el servicio con el driver de Chrome
    service = Service(ChromeDriverManager().install())
    # Inicializar el driver de Chrome con el servicio y las opciones
    driver = webdriver.Chrome(service=service, options=options)

    return driver

def create_response():
    url = "https://www.smv.gob.pe/SIMV/Frm_InformacionFinanciera?data=A70181B60967D74090DCD93C4920AA1D769614EC12"
    response = requests.get(url)

    return response


In [65]:
def select_firm(driver, text):
    """
    Selecet the firm from the dropdown button
    text: Exact name of the firm between '' 
    """    
    time.sleep(1)
    dd_firm_sel = Select(driver.find_element(By.ID, 'MainContent_cboDenominacionSocial'))
    dd_firm_sel.select_by_visible_text(text)

In [66]:
def extract_year_value(soup):
    """
    Extract a list of the values of year of the dropdown button periodo
    output : values_year (list)
    """ 
    dd_year = soup.find("select", id="MainContent_cboAnio")
    options_year = dd_year.find_all("option")
    values_year = [option["value"] for option in options_year]
    values_year = values_year[1:]
    return values_year


In [67]:
#Function select the year value
def select_year_value(driver, year):
    """
    Select the year value from the dropdown button
    """
    time.sleep(1)
    dd_year = Select(driver.find_element(By.ID, 'MainContent_cboAnio'))
    dd_year.select_by_visible_text(year)

    

In [68]:
def select_trimestre_todos(driver):
    """
    Select trimestres = TODOS
    """
    time.sleep(1)
    #select the third dropdown "todos"
    dd_trimestre = Select(driver.find_element(By.ID, 'MainContent_cboTrimestre'))

    #Select trimestre: "TODOS"
    dd_trimestre.select_by_visible_text('TODOS')

   

In [69]:
# Function Click Buscar
def click_buscar(driver):
    '''
    goes and click in "Buscar" after selecting the options
    '''
    time.sleep(1)
    a_buscar = driver.find_element(By.ID, 'MainContent_cbBuscar')
    a_buscar.click()
    time.sleep(6)

In [70]:
def parse_table(driver):
    """
    Parse with BeutifulSoup the table that appears below
    """
    time.sleep(1)
    page_source0 = driver.page_source
    soup0 = BeautifulSoup(page_source0, 'html.parser')
    return soup0

In [71]:
def find_EF(table, tri):
    """
    Find Estado Financieros in the table of the row of the trimestre
    """
    time.sleep(1)
    target_row = None

    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if len(cells) >= 2 and cells[0].text.strip() == tri and cells[2].text.strip() == 'Estado Financieros':
            target_row = row
            break
    return target_row

In [72]:
#list of trimestres
list_keys_trimestres = ['I', 'II', 'III', 'IV']

In [73]:
def find_link_click_switch(driver, target_row):
    """
    In the target row of trimestre and Estado Financiero, find the link, click the icon and switch the window
    """
    time.sleep(1)
    link_element = target_row.find('a')
    id_link_element = link_element.get('id')
    
    time.sleep(1)
    a_estadosfinancieros1 = driver.find_element(By.ID, id_link_element)
    a_estadosfinancieros1.click()

    # Switch the WebDriver instance to the new page
    driver.switch_to.window(driver.window_handles[-1])

In [74]:
def click_estado_financiero(driver, xpath):
    """
    Click in financial document"
    xpath = (String) Xpath of the document
    """
    time.sleep(2)
    a_documento = driver.find_element(By.XPATH, xpath)
    a_documento.click()

In [75]:
def find_table_tr_elements(soup, id_table):
    """
    After parser html with BS find the table and find all the tr_elements
    id_table = id of the table 
    soup = object created after parser html     
    """
    table = soup.find("table", id=id_table)
    tr_elements = table.find_all("tr")
    return tr_elements

In [76]:
items_balance = [
    "Total Activos Corrientes",
    "Total Activos No Corrientes",
    "Total Pasivos Corrientes",
    "Total Pasivos No Corrientes",
    "Total Patrimonio" 
]

items_resultados = [
    "Ingresos de Actividades Ordinarias",
    "Ganancia (Pérdida) Bruta",
    "Ganancia (Pérdida) Operativa",
    "Ganancia (Pérdida) antes de Impuestos",
    "Ganancia (Pérdida) Neta del Ejercicio"
]

items_flujo_efectivo = [
    "Flujos de Efectivo y Equivalente al Efectivo procedente de (utilizados en) Actividades de Operación",
    "Flujos de Efectivo y Equivalente al Efectivo procedente de (utilizados en) Actividades de Inversión",
    "Flujos de Efectivo y Equivalente al Efectivo procedente de (utilizados en) Actividades de Financiación",
    "Efectivo y Equivalente al Efectivo al Finalizar el Ejercicio"
]

In [77]:
def extract_cuentas_valor(items, tr_elements):
    """
    Extract cuentas and valor of each the items (Cuentas)
    items = list of items (cuentas)
    tr_elements = tr elements after finding all the tr elements of the table parsered 
    return = {items, valor}
    """
    valor = []
    # Loop through the table rows
    for item_name in items:
        for tr in tr_elements:
            td_elements = tr.find_all("td")
            if len(td_elements) > 2 and item_name == td_elements[0].text.strip():
                valor.append(td_elements[2].text.strip())

    return {'cuentas': items, 'valor': valor}

In [78]:
def concat_df_by_tri(df_concatenated, dict_cuentas_valor, tri, year):
    """
    Concat the dfs generated by trimestre
    dict_cuentas_valor = Dictionary cuentas:valor extracted
    return = df concatenated
    """
    #concat the dfs
    if tri == 'I':
        df_concatenated = pd.DataFrame(dict_cuentas_valor)
        df_concatenated['trimestre'] = tri
        df_concatenated['year'] = year
    else:
        temp = pd.DataFrame(dict_cuentas_valor)
        temp['trimestre'] = tri
        temp['year'] = year
        df_concatenated = pd.concat([df_concatenated, temp])
    
    return df_concatenated

In [88]:
df = pd.DataFrame({"cuentas": ["a", "b"], "valor": [1, 0]})

In [89]:
df.head()

,cuentas,valor
0,a,1
1,b,0


In [79]:
def switch_previous_page(driver):
    
    # Switch the WebDriver instance to the previous page
    driver.switch_to.window(driver.window_handles[-1])
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [80]:
def concat_df_by_year(df_final_concatenated, df_concatenated_year, values_year, year):    
    #concat the tables generated for each year
    if year == values_year[0]:
        df_final_concatenated = df_concatenated_year.copy()
    else:
        temp = df_concatenated_year.copy()
        df_final_concatenated = pd.concat([df_final_concatenated, temp])
    return df_final_concatenated

In [110]:
values_year = []
for i in range(2017, 2024+1):
    values_year.append(str(i))

In [112]:
def extract_firm_years(firm, list_keys_trimestres, items_balance, items_flujo_efectivo, items_resultados, values_year):
    """
    Webscrapping by name of firm and list of years and names of cuentas of balance general, estado resultados and flujo efectivo
    Note: names of cuentas change before 2017
    firm = string name of firm
    list_keys_trimestre = list with the trimestres in capital letters
    items_balance = list of selected names of cuentas
    items_flujo_efectivo = list of selected names of cuentas
    items_resultados = list of selected names of cuentas
    values_year = list of years
    """
    driver = create_driver()
    url = "https://www.smv.gob.pe/SIMV/Frm_InformacionFinanciera?data=A70181B60967D74090DCD93C4920AA1D769614EC12"
    #get the web
    driver.get(url)
    #Parse the HTML content
    #soup = BeautifulSoup(response.content, "html.parser")

    df_balance = pd.DataFrame()
    df_flujo = pd.DataFrame()
    df_resultados = pd.DataFrame()

    for year in values_year:
        print(year)
        
        select_firm(driver, firm)
        select_year_value(driver, year)
        select_trimestre_todos(driver)
        click_buscar(driver)

        df_flujo_year = pd.DataFrame()
        df_balance_year = pd.DataFrame()
        df_resultados_year = pd.DataFrame()

        for tri in list_keys_trimestres:
            print(tri)
            soup0 = parse_table(driver)
            #Find the table element with ID 'MainContent_grdInfoFinanciera'
            table = soup0.find('table', id='MainContent_grdInfoFinanciera')
            target_row = find_EF(table, tri)
        
            try:
                find_link_click_switch(driver, target_row)
                click_estado_financiero(driver, '//*[@id="lnkBalance"]')
                soup_balance = parse_table(driver)
                tr_elements_balance = find_table_tr_elements(soup_balance, "grdBalanceGeneral")
                dict_balance = extract_cuentas_valor(items_balance, tr_elements_balance)
                print(dict_balance)
                df_balance_year = concat_df_by_tri(df_balance_year, dict_balance, tri, year)
                    
                # Go to Estado de Resultados
                click_estado_financiero(driver, '//*[@id="lnkEstadoGP"]')
                soup_estado = parse_table(driver)
                tr_elements_resultados = find_table_tr_elements(soup_estado, "grdEstadoGP")
                dict_resultados = extract_cuentas_valor(items_resultados, tr_elements_resultados)
                print(dict_resultados)
                df_resultados_year = concat_df_by_tri(df_resultados_year, dict_resultados, tri, year)

                # Go to Flujo Efectivo
                click_estado_financiero(driver, '//*[@id="lnkEstadoFlujo"]')
                soup_flujo = parse_table(driver)
                tr_elements_flujo = find_table_tr_elements(soup_flujo, "grdEstadoFlujo")
                dict_flujo = extract_cuentas_valor(items_flujo_efectivo, tr_elements_flujo)
                df_flujo_year = concat_df_by_tri(df_flujo_year, dict_flujo, tri, year)
                switch_previous_page(driver)

            except AttributeError:
                pass
            
            except ElementNotInteractableException:
                switch_previous_page(driver)
                
            except ElementClickInterceptedException:
                pass

        df_balance = concat_df_by_year(df_balance, df_balance_year, values_year, year)
        df_flujo = concat_df_by_year(df_flujo, df_flujo_year, values_year, year)
        df_resultados = concat_df_by_year(df_resultados, df_resultados_year, values_year, year)

    return [df_balance, df_flujo, df_resultados]


In [113]:
list_dfs = extract_firm_years('ALICORP S.A.A.', list_keys_trimestres, items_balance, items_flujo_efectivo, items_resultados, values_year=values_year)

2017
I
{'cuentas': ['Total Activos Corrientes', 'Total Activos No Corrientes', 'Total Pasivos Corrientes', 'Total Pasivos No Corrientes', 'Total Patrimonio'], 'valor': ['1,526,262', '3,934,941', '1,795,865', '1,177,396', '2,487,942']}
{'cuentas': ['Ingresos de Actividades Ordinarias', 'Ganancia (Pérdida) Bruta', 'Ganancia (Pérdida) Operativa', 'Ganancia (Pérdida) antes de Impuestos', 'Ganancia (Pérdida) Neta del Ejercicio'], 'valor': ['940,883', '305,699', '98,145', '111,239', '81,907']}
II
{'cuentas': ['Total Activos Corrientes', 'Total Activos No Corrientes', 'Total Pasivos Corrientes', 'Total Pasivos No Corrientes', 'Total Patrimonio'], 'valor': ['1,673,823', '3,930,084', '1,815,976', '1,175,558', '2,612,373']}
{'cuentas': ['Ingresos de Actividades Ordinarias', 'Ganancia (Pérdida) Bruta', 'Ganancia (Pérdida) Operativa', 'Ganancia (Pérdida) antes de Impuestos', 'Ganancia (Pérdida) Neta del Ejercicio'], 'valor': ['1,064,539', '344,790', '119,376', '156,193', '126,295']}
III
{'cuentas'

In [116]:
list_dfs[0].to_csv('df_balance.csv')

In [117]:
list_dfs[1].to_csv('df_flujo.csv')

In [118]:
list_dfs[2].to_csv('df_resultados.csv')